# 1-D Stencil using CUDA

CEPARCO SIMD Deep Dive programming project.

#### Author:
- HERNANDEZ, Pierre Vincent

## **`2^20`**

In [ ]:
%%writefile stencil_CUDA_2-20.cu
// Name: HERNANDEZ, Pierre Vincent C.		CEPARCO - S11
// About: 1-D Stencil using CUDA
// Number of elements: 2^20

#include <stdio.h>
#include <stdlib.h>


// CUDA init kernel
__global__
void stencil_1D_CUDA(long long int* Y, long long int* X, int n) {
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i > 2 && i < n - 3; i += stride) {
    Y[i] = X[i - 3] + X[i - 2] + X[i - 1] + X[i] + X[i + 1] + X[i + 2] + X[i + 3];
  }
}


// CUDA init kernel
__global__
void initY(int n, long long int* Y) {
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < n; i += stride) {
    Y[i] = 0L;
  }
}


int main(void) {
  const int n = 1<<20;
  const long long int VECTOR_BYTES = n * sizeof(long long int);
  const int RUN_COUNT = 30;

  // Allocate memory for vectors
	long long int *X, *Y;
  cudaMallocManaged(&X, VECTOR_BYTES);
  cudaMallocManaged(&Y, VECTOR_BYTES);

  // Display current specs
	printf("\n=== 1-D Stencil ===\n");
	printf("Number of elements (n):  %d\n", n);
	printf("Byte size per vector:  %lld\n", VECTOR_BYTES);
	printf("Number of runs:  %d\n\n", RUN_COUNT);

  // Setup number of GPU threads and blocks to be used
  int numThreads = 1024; // maximize threads
  int numBlocks = (n + numThreads - 1) / numThreads;
  printf("Num. of Threads = %d\nNum. of Blocks = %d\n", numThreads, numBlocks);

  // Setup where X vector is going to be located
  int device = -1;
  cudaGetDevice(&device); // get device number of GPU
  printf("Device # = %d\n", device);
  // Advise GPU that the `X` vector stays in the host memory -- copy and paste
  cudaMemAdvise(X, VECTOR_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  // Asynchronously transfer data ahead of time from GPU memory back to the host memory
  cudaMemPrefetchAsync(X, VECTOR_BYTES, cudaCpuDeviceId, NULL); // NULL means run one stream
  // Asynchronously transfer data ahead of time to the GPU memory -- Cut and paste
  cudaMemPrefetchAsync(Y, VECTOR_BYTES, device, NULL); // NULL means run one stream

  //init data
  for(int i = 0; i < n; i++){
    X[i] = (long long int)(i + 1);
  }
  initY <<<numBlocks, numThreads>>> (n, Y);

  // Advise GPU that the `X` array is read only
  cudaMemAdvise(X, VECTOR_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  // Copy and paste data from host memory to GPU memory
  cudaMemPrefetchAsync(X, VECTOR_BYTES, device, NULL); // NULL means run one stream


  // Run stencil kernel many times
  for(int i = 0; i < RUN_COUNT; i++) {
    stencil_1D_CUDA <<<numBlocks, numThreads>>> (Y, X, n);
  }
  //wait to return from GPU
  cudaDeviceSynchronize();

  // Asynchronously transfer data ahead of time from GPU memory back to the host memory
  cudaMemPrefetchAsync(Y, VECTOR_BYTES, cudaCpuDeviceId, NULL); // NULL means run one stream

  //check for errors
  unsigned int err_count = 0;
  for(int i = 3; i < n - 3; i++) {
    if(Y[i] != (X[i - 3] + X[i - 2] + X[i - 1] + X[i] + X[i + 1] + X[i + 2] + X[i + 3]))
      ++err_count;
  }
  printf("Error count (CUDA program): %d \n\n", err_count);

  // Free memory
  cudaFree(X);
  cudaFree(Y);

  return 0;
}


Writing stencil_CUDA_2-20.cu


In [ ]:
%%shell
nvcc stencil_CUDA_2-20.cu -o stencil_CUDA_2-20

In [ ]:
%%shell
nvprof ./stencil_CUDA_2-20

==913== NVPROF is profiling process 913, command: ./stencil_CUDA_2-20

=== 1-D Stencil ===
Number of elements (n):  1048576
Byte size per vector:  8388608
Number of runs:  30

Num. of Threads = 1024
Num. of Blocks = 1024
Device # = 0
Error count (CUDA program): 0 

==913== Profiling application: ./stencil_CUDA_2-20
==913== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   98.34%  2.9165ms        30  97.215us  94.303us  98.848us  stencil_1D_CUDA(__int64*, __int64*, int)
                    1.66%  49.087us         1  49.087us  49.087us  49.087us  initY(int, __int64*)
      API calls:   97.63%  423.56ms         2  211.78ms  50.003us  423.51ms  cudaMallocManaged
                    1.09%  4.7476ms         4  1.1869ms  200.55us  2.7448ms  cudaMemPrefetchAsync
                    0.74%  3.1991ms         1  3.1991ms  3.1991ms  3.1991ms  cudaDeviceSynchronize
                    0.35%  1.5109ms         2  755.47us  699.63us  

## **`2^24`**

In [ ]:
%%writefile stencil_CUDA_2-24.cu
// Name: HERNANDEZ, Pierre Vincent C.		CEPARCO - S11
// About: 1-D Stencil using CUDA
// Number of elements: 2^24

#include <stdio.h>
#include <stdlib.h>


// CUDA init kernel
__global__
void stencil_1D_CUDA(long long int* Y, long long int* X, int n) {
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i > 2 && i < n - 3; i += stride) {
    Y[i] = X[i - 3] + X[i - 2] + X[i - 1] + X[i] + X[i + 1] + X[i + 2] + X[i + 3];
  }
}


// CUDA init kernel
__global__
void initY(int n, long long int* Y) {
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < n; i += stride) {
    Y[i] = 0L;
  }
}


int main(void) {
  const int n = 1<<24;
  const long long int VECTOR_BYTES = n * sizeof(long long int);
  const int RUN_COUNT = 30;

  // Allocate memory for vectors
	long long int *X, *Y;
  cudaMallocManaged(&X, VECTOR_BYTES);
  cudaMallocManaged(&Y, VECTOR_BYTES);

  // Display current specs
	printf("\n=== 1-D Stencil ===\n");
	printf("Number of elements (n):  %d\n", n);
	printf("Byte size per vector:  %lld\n", VECTOR_BYTES);
	printf("Number of runs:  %d\n\n", RUN_COUNT);

  // Setup number of GPU threads and blocks to be used
  int numThreads = 1024; // maximize threads
  int numBlocks = (n + numThreads - 1) / numThreads;
  printf("Num. of Threads = %d\nNum. of Blocks = %d\n", numThreads, numBlocks);

  // Setup where X vector is going to be located
  int device = -1;
  cudaGetDevice(&device); // get device number of GPU
  printf("Device # = %d\n", device);
  // Advise GPU that the `X` vector stays in the host memory -- copy and paste
  cudaMemAdvise(X, VECTOR_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  // Asynchronously transfer data ahead of time from GPU memory back to the host memory
  cudaMemPrefetchAsync(X, VECTOR_BYTES, cudaCpuDeviceId, NULL); // NULL means run one stream
  // Asynchronously transfer data ahead of time to the GPU memory -- Cut and paste
  cudaMemPrefetchAsync(Y, VECTOR_BYTES, device, NULL); // NULL means run one stream

  //init data
  for(int i = 0; i < n; i++){
    X[i] = (long long int)(i + 1);
  }
  initY <<<numBlocks, numThreads>>> (n, Y);

  // Advise GPU that the `X` array is read only
  cudaMemAdvise(X, VECTOR_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  // Copy and paste data from host memory to GPU memory
  cudaMemPrefetchAsync(X, VECTOR_BYTES, device, NULL); // NULL means run one stream


  // Run stencil kernel many times
  for(int i = 0; i < RUN_COUNT; i++) {
    stencil_1D_CUDA <<<numBlocks, numThreads>>> (Y, X, n);
  }
  //wait to return from GPU
  cudaDeviceSynchronize();

  // Asynchronously transfer data ahead of time from GPU memory back to the host memory
  cudaMemPrefetchAsync(Y, VECTOR_BYTES, cudaCpuDeviceId, NULL); // NULL means run one stream

  //check for errors
  unsigned int err_count = 0;
  for(int i = 3; i < n - 3; i++) {
    if(Y[i] != (X[i - 3] + X[i - 2] + X[i - 1] + X[i] + X[i + 1] + X[i + 2] + X[i + 3]))
      ++err_count;
  }
  printf("Error count (CUDA program): %d \n\n", err_count);

  // Free memory
  cudaFree(X);
  cudaFree(Y);

  return 0;
}


Writing stencil_CUDA_2-24.cu


In [ ]:
%%shell
nvcc stencil_CUDA_2-24.cu -o stencil_CUDA_2-24

In [ ]:
%%shell
nvprof ./stencil_CUDA_2-24

==991== NVPROF is profiling process 991, command: ./stencil_CUDA_2-24

=== 1-D Stencil ===
Number of elements (n):  16777216
Byte size per vector:  134217728
Number of runs:  30

Num. of Threads = 1024
Num. of Blocks = 16384
Device # = 0
Error count (CUDA program): 0 

==991== Profiling application: ./stencil_CUDA_2-24
==991== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   98.42%  44.254ms        30  1.4751ms  1.4704ms  1.4827ms  stencil_1D_CUDA(__int64*, __int64*, int)
                    1.58%  712.29us         1  712.29us  712.29us  712.29us  initY(int, __int64*)
      API calls:   60.80%  245.35ms         2  122.68ms  63.185us  245.29ms  cudaMallocManaged
                   17.83%  71.958ms         4  17.989ms  994.77us  44.498ms  cudaMemPrefetchAsync
                   12.43%  50.146ms         1  50.146ms  50.146ms  50.146ms  cudaDeviceSynchronize
                    6.34%  25.601ms         2  12.801ms  12.791

## **`2^28`**

In [ ]:
%%writefile stencil_CUDA_2-28.cu
// Name: HERNANDEZ, Pierre Vincent C.		CEPARCO - S11
// About: 1-D Stencil using CUDA
// Number of elements: 2^28

#include <stdio.h>
#include <stdlib.h>


// CUDA init kernel
__global__
void stencil_1D_CUDA(long long int* Y, long long int* X, int n) {
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i > 2 && i < n - 3; i += stride) {
    Y[i] = X[i - 3] + X[i - 2] + X[i - 1] + X[i] + X[i + 1] + X[i + 2] + X[i + 3];
  }
}


// CUDA init kernel
__global__
void initY(int n, long long int* Y) {
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < n; i += stride) {
    Y[i] = 0L;
  }
}


int main(void) {
  const int n = 1<<28;
  const long long int VECTOR_BYTES = n * sizeof(long long int);
  const int RUN_COUNT = 30;

  // Allocate memory for vectors
	long long int *X, *Y;
  cudaMallocManaged(&X, VECTOR_BYTES);
  cudaMallocManaged(&Y, VECTOR_BYTES);

  // Display current specs
	printf("\n=== 1-D Stencil ===\n");
	printf("Number of elements (n):  %d\n", n);
	printf("Byte size per vector:  %lld\n", VECTOR_BYTES);
	printf("Number of runs:  %d\n\n", RUN_COUNT);

  // Setup number of GPU threads and blocks to be used
  int numThreads = 1024; // maximize threads
  int numBlocks = (n + numThreads - 1) / numThreads;
  printf("Num. of Threads = %d\nNum. of Blocks = %d\n", numThreads, numBlocks);

  // Setup where X vector is going to be located
  int device = -1;
  cudaGetDevice(&device); // get device number of GPU
  printf("Device # = %d\n", device);
  // Advise GPU that the `X` vector stays in the host memory -- copy and paste
  cudaMemAdvise(X, VECTOR_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  // Asynchronously transfer data ahead of time from GPU memory back to the host memory
  cudaMemPrefetchAsync(X, VECTOR_BYTES, cudaCpuDeviceId, NULL); // NULL means run one stream
  // Asynchronously transfer data ahead of time to the GPU memory -- Cut and paste
  cudaMemPrefetchAsync(Y, VECTOR_BYTES, device, NULL); // NULL means run one stream

  //init data
  for(int i = 0; i < n; i++){
    X[i] = (long long int)(i + 1);
  }
  initY <<<numBlocks, numThreads>>> (n, Y);

  // Advise GPU that the `X` array is read only
  cudaMemAdvise(X, VECTOR_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  // Copy and paste data from host memory to GPU memory
  cudaMemPrefetchAsync(X, VECTOR_BYTES, device, NULL); // NULL means run one stream


  // Run stencil kernel many times
  for(int i = 0; i < RUN_COUNT; i++) {
    stencil_1D_CUDA <<<numBlocks, numThreads>>> (Y, X, n);
  }
  //wait to return from GPU
  cudaDeviceSynchronize();

  // Asynchronously transfer data ahead of time from GPU memory back to the host memory
  cudaMemPrefetchAsync(Y, VECTOR_BYTES, cudaCpuDeviceId, NULL); // NULL means run one stream

  //check for errors
  unsigned int err_count = 0;
  for(int i = 3; i < n - 3; i++) {
    if(Y[i] != (X[i - 3] + X[i - 2] + X[i - 1] + X[i] + X[i + 1] + X[i + 2] + X[i + 3]))
      ++err_count;
  }
  printf("Error count (CUDA program): %d \n\n", err_count);

  // Free memory
  cudaFree(X);
  cudaFree(Y);

  return 0;
}


Writing stencil_CUDA_2-28.cu


In [ ]:
%%shell
nvcc stencil_CUDA_2-28.cu -o stencil_CUDA_2-28

In [ ]:
%%shell
nvprof ./stencil_CUDA_2-28

==1079== NVPROF is profiling process 1079, command: ./stencil_CUDA_2-28

=== 1-D Stencil ===
Number of elements (n):  268435456
Byte size per vector:  2147483648
Number of runs:  30

Num. of Threads = 1024
Num. of Blocks = 262144
Device # = 0
Error count (CUDA program): 0 

==1079== Profiling application: ./stencil_CUDA_2-28
==1079== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   97.90%  528.99ms        30  17.633ms  17.445ms  20.853ms  stencil_1D_CUDA(__int64*, __int64*, int)
                    2.10%  11.326ms         1  11.326ms  11.326ms  11.326ms  initY(int, __int64*)
      API calls:   52.75%  1.34181s         4  335.45ms  14.713ms  662.67ms  cudaMemPrefetchAsync
                   20.79%  528.90ms         1  528.90ms  528.90ms  528.90ms  cudaDeviceSynchronize
                   12.79%  325.37ms         2  162.69ms  156.63ms  168.74ms  cudaFree
                    9.54%  242.70ms         2  121.35ms  49.057us

## **`2^30`**

In [ ]:
%%writefile stencil_CUDA_2-30.cu
// Name: HERNANDEZ, Pierre Vincent C.		CEPARCO - S11
// About: 1-D Stencil using CUDA
// Number of elements: 2^30

#include <stdio.h>
#include <stdlib.h>


// CUDA init kernel
__global__
void stencil_1D_CUDA(long long int* Y, long long int* X, int n) {
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i > 2 && i < n - 3; i += stride) {
    Y[i] = X[i - 3] + X[i - 2] + X[i - 1] + X[i] + X[i + 1] + X[i + 2] + X[i + 3];
  }
}


// CUDA init kernel
__global__
void initY(int n, long long int* Y) {
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < n; i += stride) {
    Y[i] = 0L;
  }
}


int main(void) {
  const int n = 1<<30;
  const long long int VECTOR_BYTES = n * sizeof(long long int);
  const int RUN_COUNT = 30;

  // Allocate memory for vectors
	long long int *X, *Y;
  cudaMallocManaged(&X, VECTOR_BYTES);
  cudaMallocManaged(&Y, VECTOR_BYTES);

  // Display current specs
	printf("\n=== 1-D Stencil ===\n");
	printf("Number of elements (n):  %d\n", n);
	printf("Byte size per vector:  %lld\n", VECTOR_BYTES);
	printf("Number of runs:  %d\n\n", RUN_COUNT);

  // Setup number of GPU threads and blocks to be used
  int numThreads = 1024; // maximize threads
  int numBlocks = (n + numThreads - 1) / numThreads;
  printf("Num. of Threads = %d\nNum. of Blocks = %d\n", numThreads, numBlocks);

  // Setup where X vector is going to be located
  int device = -1;
  cudaGetDevice(&device); // get device number of GPU
  printf("Device # = %d\n", device);
  // Advise GPU that the `X` vector stays in the host memory -- copy and paste
  cudaMemAdvise(X, VECTOR_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  // Asynchronously transfer data ahead of time from GPU memory back to the host memory
  cudaMemPrefetchAsync(X, VECTOR_BYTES, cudaCpuDeviceId, NULL); // NULL means run one stream
  // Asynchronously transfer data ahead of time to the GPU memory -- Cut and paste
  cudaMemPrefetchAsync(Y, VECTOR_BYTES, device, NULL); // NULL means run one stream

  //init data
  for(int i = 0; i < n; i++){
    X[i] = (long long int)(i + 1);
  }
  initY <<<numBlocks, numThreads>>> (n, Y);

  // Advise GPU that the `X` array is read only
  cudaMemAdvise(X, VECTOR_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  // Copy and paste data from host memory to GPU memory
  cudaMemPrefetchAsync(X, VECTOR_BYTES, device, NULL); // NULL means run one stream


  // Run stencil kernel many times
  for(int i = 0; i < RUN_COUNT; i++) {
    stencil_1D_CUDA <<<numBlocks, numThreads>>> (Y, X, n);
  }
  //wait to return from GPU
  cudaDeviceSynchronize();

  // Asynchronously transfer data ahead of time from GPU memory back to the host memory
  cudaMemPrefetchAsync(Y, VECTOR_BYTES, cudaCpuDeviceId, NULL); // NULL means run one stream

  //check for errors
  unsigned int err_count = 0;
  for(int i = 3; i < n - 3; i++) {
    if(Y[i] != (X[i - 3] + X[i - 2] + X[i - 1] + X[i] + X[i + 1] + X[i + 2] + X[i + 3]))
      ++err_count;
  }
  printf("Error count (CUDA program): %d \n\n", err_count);

  // Free memory
  cudaFree(X);
  cudaFree(Y);

  return 0;
}


Writing stencil_CUDA_2-30.cu


In [ ]:
%%shell
nvcc stencil_CUDA_2-30.cu -o stencil_CUDA_2-30

In [ ]:
%%shell
nvprof ./stencil_CUDA_2-30

==1664== NVPROF is profiling process 1664, command: ./stencil_CUDA_2-30

=== 1-D Stencil ===
Number of elements (n):  1073741824
Byte size per vector:  8589934592
Number of runs:  30

Num. of Threads = 1024
Num. of Blocks = 1048576
Device # = 0
